1. Implement the "user-based similarity" method of Collaborative Filtering. Upload the "ratings.csv". Create a frame for this file. 

In [3]:
from google.colab import files
uploaded = files.upload()


Saving rating.csv to rating.csv


In [4]:
import pandas as pd
df = pd.read_csv("/content/rating.csv")
df.head(5)

,userId,movieId,rating,timestamp
0,1,2,3.5,02-04-2005 23:53
1,1,29,3.5,02-04-2005 23:31
2,1,32,3.5,02-04-2005 23:33
3,1,47,3.5,02-04-2005 23:32
4,1,50,3.5,02-04-2005 23:29


2. Drop the timestamp column and find the number of unique movies and unique users.

In [5]:
df.drop('timestamp', axis = 1, inplace = True) 
print("No. of unique users =",len(df['userId'].unique()))
print("No. of unique movies =",len(df['movieId'].unique()))


No. of unique users = 7120
No. of unique movies = 14026


3. Create a pivot table or matrix with users as rows and movies as columns. Matrix entries will represent the ratings given by the users. This will be a sparse matrix and those movies not watched will be NaN. Replace all such NaN values by zeros. 

In [ ]:
user_movies_df = df.pivot(index = "userId", columns = "movieId", values = "rating").reset_index(drop = True)
user_movies_df.index = df.userId.unique()
user_movies_df.fillna(0, inplace = True)


4. Compute the cosine similarity matrix between all pairs of users.The diagonal values of this matrix will be 1. Print the matrix.

In [ ]:
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation
user_sim = 1 - pairwise_distances(user_movies_df.values, metric = "cosine") 
#covert this matrix to a frame
user_sim_df = pd.DataFrame(user_sim)
#set the index and column names to user ids 
user_sim_df.index = df.userId.unique()
user_sim_df.columns = df.userId.unique()
user_sim_df.iloc[0:5, 0:5]

,1,2,3,4,5
1,1.000000,0.102916,0.261198,0.029091,0.139199
2,0.102916,1.000000,0.180124,0.064014,0.140042
3,0.261198,0.180124,1.000000,0.057323,0.156755
4,0.029091,0.064014,0.057323,1.000000,0.300828
5,0.139199,0.140042,0.156755,0.300828,1.000000


5. Set the diagonal values as 0.0, since we need to find other users who are similar to a specific user.

In [ ]:
import numpy as np
np.fill_diagonal(user_sim, 0)
user_sim_df.iloc[0:5, 0:5]

,1,2,3,4,5
1,0.000000,0.102916,0.261198,0.029091,0.139199
2,0.102916,0.000000,0.180124,0.064014,0.140042
3,0.261198,0.180124,0.000000,0.057323,0.156755
4,0.029091,0.064014,0.057323,0.000000,0.300828
5,0.139199,0.140042,0.156755,0.300828,0.000000


6. Filter similar users. To find most similar users, the maximum values of each column can be filtered. For example, the most similar user to first 5 users with userId from 1 to 5 can be printed as

In [ ]:
user_sim_df.idxmax(axis=1)[0:5]

1    2595
2    5964
3     475
4     242
5    1367
dtype: int64

7. This result says that the most similar user to user1 is userId 2595 and so on.This also means that both have watched several movies in common and rated very similarly. This can be verified with the movies dataset. Load the movies dataset, drop the "genres" column.


In [ ]:
uploaded = files.upload()

Saving movie.csv to movie.csv


In [ ]:
movies_df = pd.read_csv("/content/movie.csv")
movies_df.drop('genres', axis = 1, inplace = True)
movies_df.iloc[0:5, 0:5]

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


8. Find common movies of similar users. 

In [ ]:
def get_user_similar_movies(user1, user2):
  #Inner join of movies watched by both users will give the common movies
  common = df[df.userId == user1].merge(df[df.userId == user2], on = "movieId", how = "inner")
  #merge this result with movie details
  return common.merge(movies_df, on = "movieId")


9. Find the common movies of user 1 and user 2595.Print only those common movies with ratings above 4.

In [ ]:
common_movies = get_user_similar_movies(1,2595 )
#print(common_movies)
common_movies[(common_movies.rating_x >=4.0) & ((common_movies.rating_y >=4.0))]

,userId_x,movieId,rating_x,userId_y,rating_y,title
3,1,260,4.0,2595,5.0,Star Wars: Episode IV - A New Hope (1977)
4,1,293,4.0,2595,4.0,Léon: The Professional (a.k.a. The Professiona...
5,1,296,4.0,2595,5.0,Pulp Fiction (1994)
6,1,318,4.0,2595,5.0,"Shawshank Redemption, The (1994)"
7,1,541,4.0,2595,4.5,Blade Runner (1982)
12,1,1036,4.0,2595,4.5,Die Hard (1988)
13,1,1097,4.0,2595,5.0,E.T. the Extra-Terrestrial (1982)
16,1,1196,4.5,2595,4.5,Star Wars: Episode V - The Empire Strikes Back...
17,1,1198,4.5,2595,5.0,Raiders of the Lost Ark (Indiana Jones and the...
18,1,1200,4.0,2595,4.5,Aliens (1986)


10. Finding user similarity does not work for new users. We need to wait until the new user buys a few items and rates them. Only then user preferences can be founnd and recommendations can be made. This is called the "cold-start" problem in recommender systems. This is overcome by using item-based similarity methods. 